In [1]:
import django
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'fbpost.settings')
django.setup()

In [2]:
from aggregation.models import Author ,Publisher ,Book ,Store

In [178]:
a1=Author(name="author1",age=25)
a1.save()
p1=Publisher(name="pub1")
p1.save()
b1=Book(name="book1",pages=50,price=43.8,rating=3.5,publisher=p1)
b1.authors.add(a1)
b1.save()

ValueError: "<Book: Book object (None)>" needs to have a value for field "id" before this many-to-many relationship can be used.

In [8]:
b1.authors.name

In [9]:
Book.objects.count()

1

In [10]:
b2=Book(name="book2",pages=50,price=33.6,rating=4.5,publisher=p1)
b2.save()

In [12]:
Book.objects.count()

2

In [172]:
from django.db.models import Max ,Avg,FloatField ,Min
Book.objects.aggregate(Avg('price'))

{'price__avg': Decimal('38.7000000000000')}

In [18]:
Book.objects.aggregate(Max('rating'))

{'rating__max': 4.5}

In [32]:
from django.db.models import FloatField
Book.objects.aggregate(price_diff=Max('price') - Avg('price'))

{'price_diff': Decimal('5.09999999999999')}

In [33]:
from django.db.models import Count

In [122]:
pubs=Publisher.objects.annotate(num_books=Count('book'))

In [125]:
pubs[1].num_books

2

In [55]:
from django.db.models import Q ,F

In [143]:
above_3= Count('book', filter=Q(book__rating__gt=3))

In [144]:
below_3=Count('book',filter=Q(book__rating__lte=3))

In [145]:
below_3

Count(F(book), filter=(AND: ('book__rating__lte', 3)))

In [158]:
pubs = Publisher.objects.annotate(above_3=above_3)
Publisher.objects.annotate(below_3=below_3)

<QuerySet [<Publisher: Publisher object (2)>]>

In [159]:
pubs[0].above_3

2

In [134]:
Book.objects.all()[1].rating

4.5

In [128]:
Publisher.objects.all()

<QuerySet [<Publisher: Publisher object (2)>]>

In [160]:
pubs = Publisher.objects.annotate(num_books=Count('book')).order_by('-num_books')[:5]

In [161]:
pubs

<QuerySet [<Publisher: Publisher object (2)>]>

In [164]:
pubs[0].num_books

2

In [173]:
Book.objects.aggregate(top_rating=Max('rating'),min_rating=Min('rating'))

{'top_rating': 4.5, 'min_rating': 3.5}

<QuerySet [<Book: Book object (1)>, <Book: Book object (2)>]>

In [183]:
Store.objects.all().delete()

(0, {})

In [186]:
b1=Book(name="book1",pages=50,price=43.8,rating=3.5,publisher=p1)
b1.save()

In [187]:
b1.authors.add(a1)

In [188]:
Author.objects.all()

<QuerySet [<Author: Author object (7)>]>

In [189]:
Book.objects.all()

<QuerySet [<Book: Book object (5)>]>

In [190]:
b2=Book(name="book2",pages=50,price=33.6,rating=4.5,publisher=p1)
b2.save()
b2.authors.add(a1)


In [193]:
q=Book.objects.annotate(Count('authors'))

In [197]:
q[1].authors__count

1

In [198]:
s1=Store(name="store 1")
s1.save()
s2=Store(name="store 2")
s2.save()

In [199]:
s1.books.add(b1)
s2.books.add(b2)

In [203]:
s=Store.objects.annotate(mi_price=Min('books__price'),max_price=Max('books__price'))

In [206]:
s[1].max_price

Decimal('33.6000000000000')

In [207]:
Author.objects.aggregate(average_rating=Avg('book__rating'))

{'average_rating': 4.0}

In [208]:
b=Book.objects.filter(name__startswith="book1").annotate(num_authors=Count('authors'))


In [209]:
b

<QuerySet [<Book: Book object (5)>]>

In [211]:
b.first().num_authors

1

In [212]:
Book.objects.all()

<QuerySet [<Book: Book object (5)>, <Book: Book object (6)>]>

In [214]:
Book.objects.all()[0].authors.count()

1

In [215]:
Book.objects.all()[1].authors.count()

1

In [216]:
Book.objects.all()[0].authors.all()[0].id

7

In [217]:
Book.objects.all()[1].authors.all()[0].id

7

In [218]:
a2=Author(name="author2",age =29)
a2.save()
Book.objects.all()[1].authors.add(a2)

In [220]:
Book.objects.all()[1].authors.count()

2

In [221]:
Book.objects.annotate(num_authors=Count('authors')).order_by('num_authors')

<QuerySet [<Book: Book object (5)>, <Book: Book object (6)>]>

In [222]:
Book.objects.annotate(num_authors=Count('authors')).order_by('-num_authors')

<QuerySet [<Book: Book object (6)>, <Book: Book object (5)>]>

In [226]:
Book.objects.values('name')

<QuerySet [{'name': 'book1'}, {'name': 'book2'}]>

In [227]:
Author.objects.annotate(average_rating=Avg('book__rating')).values('name', 'average_rating')


<QuerySet [{'name': 'author1', 'average_rating': 4.0}, {'name': 'author2', 'average_rating': 4.5}]>

In [228]:
Book.objects.annotate(num_authors=Count('authors')).aggregate(Avg('num_authors'))

{'num_authors__avg': 1.5}

In [229]:
Book.objects.annotate(num_authors=Count('authors')).values('num_authors')

<QuerySet [{'num_authors': 1}, {'num_authors': 2}]>

In [238]:
Store.objects.aggregate(min_price=Min('books__price'))

{'min_price': Decimal('33.6000000000000')}

In [241]:
Author.objects.filter(book__rating__gte=3).annotate(highly_rated=Count('book'))

<QuerySet [<Author: Author object (7)>, <Author: Author object (8)>]>

In [244]:
b3=Book(name="book3",pages=50,price=33.6,rating=1,publisher=p1)
b3.save()
b3.authors.add(a1)

In [245]:
Author.objects.filter(book__rating__gte=3).annotate(highly_rated=Count('book'))

<QuerySet [<Author: Author object (7)>, <Author: Author object (8)>]>

In [246]:
Author.objects.filter(book__rating__gte=3).annotate(highly_rated=Count('book')).first().highly_rated

2